In [1]:
# Імпорт необхідних бібліотек
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Підключення Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Завантаження даних з Google Drive
data_path = '/content/drive/My Drive/labsfile/mnist.npz'
(training_images, training_labels), _ = tf.keras.datasets.mnist.load_data(path=data_path)

# Попередня обробка даних
def reshape_and_normalize(images):
    # Зміна форми зображень для додавання додаткового виміру
    images = images.reshape(images.shape[0], images.shape[1], images.shape[2], 1)
    # Нормалізація значень пікселів
    images = images / 255.0
    return images

# Тестування функції
training_images = reshape_and_normalize(training_images)

# Визначення зворотного виклику
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if(logs.get('accuracy')>0.995):
            print("\nДосягнуто 99,5% точності, тому тренування скасовано!")
            self.model.stop_training = True

# Визначення моделі згорткової нейронної мережі
def convolutional_model():
    model = tf.keras.models.Sequential([
      tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(28, 28, 1)),
      tf.keras.layers.MaxPooling2D(2, 2),
      tf.keras.layers.Flatten(),
      tf.keras.layers.Dense(128, activation='relu'),
      tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Збереження ненавченої моделі на Google Drive
model_save_path = '/content/drive/My Drive/labsfile/my_model.h5'
model = convolutional_model()
callbacks = myCallback()
history = model.fit(training_images, training_labels, epochs=10, callbacks=[callbacks])
model.save(model_save_path)

# Перевірка кількості епох, протягом яких навчалася модель
print(f"Ваша модель навчалася протягом {len(history.epoch)} епох")


Mounted at /content/drive
Epoch 1/10
1875/1875 [==============================] - 37s 19ms/step - loss: 0.1481 - accuracy: 0.9554
Epoch 2/10
1875/1875 [==============================] - 37s 20ms/step - loss: 0.0495 - accuracy: 0.9851
Epoch 3/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0326 - accuracy: 0.9898
Epoch 4/10
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0204 - accuracy: 0.9937
Epoch 5/10
1875/1875 [==============================] - ETA: 0s - loss: 0.0149 - accuracy: 0.9950
Досягнуто 99,5% точності, тому тренування скасовано!
1875/1875 [==============================] - 36s 19ms/step - loss: 0.0149 - accuracy: 0.9950
Ваша модель навчалася протягом 5 епох


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
